# Econ 212: Problem Set 3

In [8]:
from fredapi import Fred
import numpy as np

In [3]:
fred = Fred(api_key = 'f6fa4f544be19c2ea791e9a3240470bf')

## Problem 1: Seignorage and inflation

## Problem 2: SVAR impulse response to monetary shocks

### Part A

In [4]:
real_GDP = fred.get_series('GDPC1')
price_deflator = fred.get_series('GDPDEF')
fed_funds_rate = fred.get_series('FEDFUNDS')

In [11]:
growth_rate = [400*np.log(real_GDP[t]/real_GDP[t-1]) for t in range(1,len(real_GDP))]
inflation_rate = [400*np.log(price_deflator[t]/price_deflator[t-1]) for t in range(1,len(price_deflator))]